In [1]:
!cat /opt/conda/lib/python3.7/site-packages/efficientnet/keras.py

cat: /opt/conda/lib/python3.7/site-packages/efficientnet/keras.py: No such file or directory


In [ ]:
submission_ending = '''from . import inject_keras_modules, init_tfkeras_custom_objects
from . import model

from .preprocessing import center_crop_and_resize

EfficientNetB0 = inject_keras_modules(model.EfficientNetB0)
EfficientNetB1 = inject_keras_modules(model.EfficientNetB1)
EfficientNetB2 = inject_keras_modules(model.EfficientNetB2)
EfficientNetB3 = inject_keras_modules(model.EfficientNetB3)
EfficientNetB4 = inject_keras_modules(model.EfficientNetB4)
EfficientNetB5 = inject_keras_modules(model.EfficientNetB5)
EfficientNetB6 = inject_keras_modules(model.EfficientNetB6)
EfficientNetB7 = inject_keras_modules(model.EfficientNetB7)

preprocess_input = inject_keras_modules(model.preprocess_input)

init_tfkeras_custom_objects()'''
with open('/opt/conda/lib/python3.7/site-packages/efficientnet/keras.py', mode='w') as file:
    file.write(submission_ending)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#!pip install opencv-python
!pip install patchify
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from PIL import Image
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import random 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Python3 code to demonstrate
# check for unique values
# Using len() + set() + values()

# initializing dictionary
test_dict = {'Manjeet': 1, 'Akash': 2, 'Punnet': 3, 'Manjeet': 4}
test_dict2 = {10: 'Manjeet', 2: 'Akash', 3: 'Dalton'}



# printing original dictionary
print("The original dictionary : " + str(test_dict))

print("The original dictionary : " + str(test_dict2))


# using len() + set() + values()
# check for unique values

#flag = len(test_dict) != len(set(test_dict.items()))
flag = len(test_dict) != len(set(test_dict.values()))
# print(set(test_dict.items()))
# print(set(test_dict2.items()))




# print result
print("Does dictionary contain repetition : " + str(flag))


The original dictionary : {'Manjeet': 4, 'Akash': 2, 'Punnet': 3}
The original dictionary : {10: 'Manjeet', 2: 'Akash', 3: 'Dalton'}
Does dictionary contain repetition : False


In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
dataset_root_folder = '/kaggle/input/semantic-segmentation-of-aerial-imagery'
dataset_name = 'Semantic segmentation dataset'

In [ ]:
for path, subdirs, files in os.walk(dataset_root_folder):
    dir_name = path.split(os.path.sep)[-1]
    #print(dir_name)
    if dir_name == 'masks': # would be images for image masks
        images = os.listdir(path)
        #print(images)
        
        
        for i, image_name in enumerate(images):
            if(image_name.endswith('.png')): # would be jpg for image tiles
                print(image_name)
            
        

In [ ]:
image_patch_size = 256

In [ ]:
image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile 2/images/image_part_005.jpg')

In [ ]:
print(image.shape)

In [ ]:
??patchify

In [ ]:
image_patches = patchify(image, (image_patch_size, image_patch_size, 3), step = image_patch_size)

In [ ]:
print((image_patches.shape))

In [ ]:
minmaxscaler.fit_transform??

In [ ]:
image_x = image_patches[0,0,:,:]
image_y = minmaxscaler.fit_transform(image_x.reshape(-1,image_x.shape[-1])).reshape(image_x.shape)
image_y[0]

#minmaxscaler = MinMaxScaler()
#minmaxscaler.fit_transform(image_x.reshape(-1,))

#MinMaxScaler

In [ ]:
image.shape

In [ ]:
type(Image.fromarray(image))

In [ ]:
(image.shape[0]//image_patch_size)*image_patch_size

In [ ]:
image_dataset = []
mask_dataset = []

for image_type in ['images', 'masks']:
    if image_type == 'images':
        image_extension = 'jpg'
    elif image_type == 'masks':
        image_extension = 'png' #could also be png
            
    for tile_id in range(1,8):
        for image_id in range(1, 20):
            image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile {tile_id}/{image_type}/image_part_00{image_id}.{image_extension}', 1)
            if image is not None:
                if image_type == 'masks':
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #print(image.shape)
                size_x = (image.shape[1]//image_patch_size)*image_patch_size
                size_y = (image.shape[0]//image_patch_size)*image_patch_size # patchify ims into multiples of 256(or 512, depends)
                #print("{} --- {} - {}".format(image.shape, size_x, size_y))
                image = Image.fromarray(image)
                image = image.crop((0, 0, size_x, size_y))
                #print("({}, {})".format(image.size[1], image.size[0])) 
                image = np.array(image)
                patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step = image_patch_size)
                #print(len(patched_images))
                for i in range(patched_images.shape[0]):
                    for j in range(patched_images.shape[1]):
                        if image_type == 'images':
                            individual_patched_image = patched_images[i,j,:,:]
                            #print(individual_patched_image.shape)
                            individual_patched_image = minmaxscaler.fit_transform(individual_patched_image.reshape(-1, individual_patched_image.shape[-1])).reshape(individual_patched_image.shape)
                            #print(individual_patched_image.shape)
                            individual_patched_image = individual_patched_image[0]
                            #print(individual_patched_image.shape)
                            image_dataset.append(individual_patched_image)
                        elif image_type == 'masks':
                            individual_patched_mask = patched_images[i,j,:,:]
                            individual_patched_mask = individual_patched_mask[0]
                            mask_dataset.append(individual_patched_mask)




In [ ]:
print(len(image_dataset))
print(len(mask_dataset))



In [ ]:
                                                 # Now let's render images in both of our datasets

In [ ]:
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)

In [ ]:
print(mask_dataset.shape)
print(image_dataset.shape)

In [ ]:
random_image_id = random.randint(0, len(image_dataset))
plt.figure(figsize=(14,8))
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])
plt.subplot(122)
plt.imshow(mask_dataset[random_image_id])

In [ ]:
 arr3d = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
 arr2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

 lower_dim_slice = arr2d[0, 1:]
 lower_dim_slice 
    


In [ ]:
                                                                    # Processing mask labels using one hot encoding

In [ ]:
class_building = '#3C1098' #hex code of the class
class_building = class_building.lstrip('#')
class_building = np.array(tuple(int(class_building[i:i+2],16)for i in (0,2,4)))
print(class_building)

class_land = '8429F6'
class_land = class_land.lstrip('#')
class_land= np.array(tuple(int(class_land[i:i+2],16)for i in (0,2,4)))
print(class_land)

class_road = '#6EC1E4'
class_road = class_road.lstrip('#')
class_road = np.array(tuple(int(class_road[i:i+2],16)for i in (0,2,4)))
print(class_road)

class_vegetation = '#FEDD3A'
class_vegetation = class_vegetation.lstrip('#')
class_vegetation = np.array(tuple(int(class_vegetation[i:i+2],16)for i in (0,2,4)))
print(class_vegetation)

class_water= '#E2A929'
class_water= class_water.lstrip('#')
class_water= np.array(tuple(int(class_water[i:i+2],16)for i in (0,2,4)))
print(class_water)

class_unlabelled= '#9B9B9B'
class_unlabelled= class_unlabelled.lstrip('#')
class_unlabelled= np.array(tuple(int(class_unlabelled[i:i+2],16)for i in (0,2,4)))
print(class_unlabelled)



In [ ]:
def rgb_to_label(label):
    label_segment = np.zeros(label.shape, dtype=np.uint8)
    label_segment[np.all(label == class_water, axis=-1)] = 0
    label_segment[np.all(label == class_land, axis=-1)] = 1
    label_segment[np.all(label == class_road, axis=-1)] = 2
    label_segment[np.all(label == class_building, axis=-1)] = 3
    label_segment[np.all(label == class_vegetation, axis=-1)] = 4
    label_segment[np.all(label == class_unlabelled, axis=-1)] = 5
    #print(label_segment.shape)
    label_segment = label_segment[:,:,0]
    print(label_segment.shape)
    return label_segment

In [ ]:
test_label= rgb_to_label(mask_dataset[500])
test_label

In [ ]:
labels = []
for i in range(mask_dataset.shape[0]):
    label = rgb_to_label(mask_dataset[i])
    labels.append(label)

In [ ]:

print(len(labels))
#print(labels)
#print(labels[0][0,0])



In [ ]:
labels = np.array(labels)
#print(labels.shape)
#print(labels[0])
labels = np.expand_dims(labels, axis=3)
#print(labels[0])
#print(labels.shape)

In [ ]:
random_image_id = random.randint(0, len(image_dataset))
plt.figure(figsize=(14,8))
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])
plt.subplot(122)
#plt.imshow(mask_dataset[random_image_id])
plt.imshow(labels[random_image_id][:,:,0])




In [ ]:
total_classes = len(np.unique(labels))

In [ ]:
total_classes

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf

In [ ]:
a = tf.keras.utils.to_categorical([0, 1, 2, 3,1,5], num_classes=6)
a = tf.constant(a, shape=[6, 6])
print(a)

In [ ]:
to_categorical??

In [ ]:
labels_categorical_dataset = to_categorical(labels, num_classes = total_classes)

In [ ]:
labels_categorical_dataset.shape

In [ ]:
master_training_dataset = image_dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(master_training_dataset, labels_categorical_dataset, test_size=0.15, random_state=100)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



In [ ]:
image_height = X_train.shape[1]
image_width = X_train.shape[2]
image_channels = X_train.shape[3]


In [ ]:
print(image_height)
print(image_width)
print(image_channels)


In [ ]:
                                             # ANDDDD THAT'S FOR THE DATA PRE PROCESSING PART NOW LET'S JUMP ON TO THE MODEL BUILDING PART

#      Building up the model

In [ ]:
!pip install -U segmentation-models

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers import concatenate, BatchNormalization, Dropout, Lambda

In [ ]:
from keras import backend as K

In [ ]:
#now we'll setup custom eval matrix, Jaccard Index/ Intersection Over Union, see https://deepai.org/machine-learning-glossary-and-terms/jaccard-index, sorry too lazy to make it a hyperlink

from keras.models

In [ ]:
def jaccard_coef(y_true, y_pred):
    y_true_flatten=K.flatten(y_true)
    y_true_pred=K.flatten(y_pred)
    intersection = K.sum(y_true_flatten * y_true_pred)
    final_coef_val = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_true_pred) - intersection + 1.0)
    return final_coef_val

In [ ]:
def multi_unet_model(n_classes=5, image_height=256, image_width=256, image_channels=1):
    inputs = Input((image_height, image_width, image_channels))
    source_input = inputs
    
    c1 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(source_input) 
    c1 = Dropout(0.2)(c1) # experiment with this one to see what maximizes the IOU coefficient
    c1 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c1)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c2)
    p2 = MaxPooling2D((2,2))(c2)
    
    c3 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c3)
    p3 = MaxPooling2D((2,2))(c3)
    
    c4 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c4)
    p4 = MaxPooling2D((2,2))(c4)
    
    c5 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c5)
   
    u6 = Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c5)
    u6 =  concatenate([u6, c4])
    c6 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c6)
    
    u7 = Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c6)
    u7 =  concatenate([u7, c3])
    c7 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c7)
    
    u8 = Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(c7)
    u8 =  concatenate([u8, c2])
    c8 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c8)
    
    u9 = Conv2DTranspose(16, (2,2), strides=(2,2), padding="same")(c8)
    u9 =  concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding ="same")(c9)
    
    outputs = Conv2D(n_classes, (1,1), activation="softmax")(c9)
    models = Model(inputs=[inputs], outputs=[outputs])
    return models

In [ ]:
metrics = ["accuracy", jaccard_coef]

In [ ]:
def get_deep_learning_model():
    return multi_unet_model(n_classes=total_classes, image_height=image_height,image_width=image_width,image_channels=image_channels)

In [ ]:
model = get_deep_learning_model()

In [ ]:
model

In [ ]:
#model.get_config()

#                Generating Loss Function

In [ ]:
weights = [0.166, 0.166, 0.166, 0.166, 0.166,0.166]

In [ ]:
submission_ending = '''from . import inject_keras_modules, init_tfkeras_custom_objects
from . import model

from .preprocessing import center_crop_and_resize

EfficientNetB0 = inject_keras_modules(model.EfficientNetB0)
EfficientNetB1 = inject_keras_modules(model.EfficientNetB1)
EfficientNetB2 = inject_keras_modules(model.EfficientNetB2)
EfficientNetB3 = inject_keras_modules(model.EfficientNetB3)
EfficientNetB4 = inject_keras_modules(model.EfficientNetB4)
EfficientNetB5 = inject_keras_modules(model.EfficientNetB5)
EfficientNetB6 = inject_keras_modules(model.EfficientNetB6)
EfficientNetB7 = inject_keras_modules(model.EfficientNetB7)

preprocess_input = inject_keras_modules(model.preprocess_input)

init_tfkeras_custom_objects()'''
with open('/opt/conda/lib/python3.7/site-packages/efficientnet/keras.py', mode='w') as file:
    file.write(submission_ending)

In [ ]:
import segmentation_models as sm

In [ ]:
#! /opt/conda/lib/python3.7/site-packages/efficientnet/keras.py

Segmentation models : using Keras as framework****

In [ ]:
dice_loss= sm.losses.DiceLoss(class_weights=weights)


In [ ]:
focal_loss= sm.losses.CategoricalFocalLoss()

In [ ]:
total_loss = dice_loss + (1 * focal_loss)

# Model Compilation

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.backend.clear_session() # setup the backend session


In [ ]:
model.compile(optimizer="adam",loss=total_loss, metrics = metrics)

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
model

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file="satellite-model-lot.png",show_shapes=True,show_layer_names=True)

In [ ]:
import keras
from IPython.display import clear_output

%matplotlib inline

In [ ]:
class PlotLoss(keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        
        
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []

        self.jaccard_coef = []
        self.val_jaccard_coef = []

        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        # self.losses.append(logs.get('loss'))
        # self.val_losses.append(logs.get('val_loss'))

        self.jaccard_coef.append(logs.get('jaccard_coef'))
        self.val_jaccard_coef.append(logs.get('val_jaccard_coef'))

        self.i += 1
    
        clear_output(wait=True)
        # plt.plot(self.x, self.losses, label="loss")
        # plt.plot(self.x, self.val_losses, label="val_loss")

        plt.plot(self.x, self.jaccard_coef, label="jaccard_coef")
        plt.plot(self.x, self.val_jaccard_coef, label="val_jaccard_coef")

        plt.legend()
        plt.show();

plot_loss = PlotLoss()
            
        

In [ ]:
model_history = model.fit(X_train, y_train, batch_size=8, verbose=1, epochs=15, validation_data=(X_test,y_test),shuffle=False)

In [ ]:
history_a = model_history

In [ ]:
loss = history_a.history['loss']
val_loss = history_a.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label="Training Loss")
plt.plot(epochs, val_loss, 'r', label="Validation Loss")
plt.title("Training Vs validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
jaccard_coef = history_a.history['jaccard_coef']
val_jaccard_coef = history_a.history['val_jaccard_coef']
epochs = range(1, len(jaccard_coef) + 1)
plt.plot(epochs, jaccard_coef, 'y', label="Training IoU")
plt.plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
plt.title("Training Vs validation IoU")
plt.xlabel("Epochs")
plt.ylabel("IoU")
plt.legend()
plt.show()

In [ ]:
model_history.params

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
len(y_pred)

In [ ]:
y_pred_argmax = np.argmax(y_pred, axis=3)

In [ ]:
y_test_argmax = np.argmax(y_test, axis=3)

# Comparing test and prediction results

In [ ]:
# we can co mpare side by side the predicted mask image and test mask image

In [ ]:
test_image_number = random.randint(0,(len(X_test)))
test_image = X_test[test_image_number]
ground_truth_image = y_test_argmax[test_image_number]

test_image_input = np.expand_dims(test_image, 0)

prediction = model.predict(test_image_input)
#prediction = saved_model.predict(test_image_input)
predicted_image = np.argmax(prediction, axis=3)


In [ ]:
predicted_image = predicted_image[0,:,:]
predicted_image


In [ ]:
plt.imshow(test_image)

In [ ]:
plt.imshow(ground_truth_image)

In [ ]:
plt.imshow(predicted_image)

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(231)
plt.title("Original Test Image")
plt.imshow(test_image)
plt.subplot(232)
plt.title("Original Masked Image")
plt.imshow(ground_truth_image)
plt.subplot(233)
plt.title("Predicted Image")
plt.imshow(predicted_image)           

In [ ]:
model.save("satellite_segmentation_full.h5")

# Model deployment and reload

In [ ]:
model.save('Satellite_image_segmentation.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model_path = '/kaggle/working/Satellite_image_segmentation.h5'

In [ ]:
ls -lah /kaggle/working/

In [ ]:
saved_model = load_model(model_path, custom_objects=({'dice_loss_plus_1focal_loss':total_loss,'jaccard_coef':jaccard_coef}))

In [ ]:
model.loss.name # very important

# Predict the mask of any image from Google Map/Earth

In [ ]:
!wget -nd -r -P /kaggle/working -A jpg https://www.allallsoft.com/gmd/support/mapviewer2.jpg

In [ ]:
image_path = '/kaggle/working/mapviewer2.jpg'

In [ ]:
plt.imshow(Image.open(image_path))

In [ ]:
image = Image.open(image_path)
image = image.resize((256,256))
image = np.array(image)
image = np.expand_dims(image,0)

In [ ]:
prediction = saved_model.predict(image)



In [ ]:
predicted_image = np.argmax(prediction, axis=3)
predicted_image = predicted_image[0,:,:]


In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(221)
plt.title("Original Test Image")
plt.imshow(Image.open(image_path))
plt.subplot(222)
plt.title("Predicted Image")
plt.imshow(predicted_image)           

# Collecting activation and gradient output from our model

In [ ]:
!pip install keract

In [ ]:
import keract as ke

In [ ]:
activations = ke.get_activations(saved_model, image,nodes_to_evaluate=None, output_format='simple',auto_compile=True,)

In [ ]:
!mkdir /kaggle/activations

In [ ]:
ls -lah /kaggle/activations

In [ ]:
ke.display_activations(activations, cmap='virdis',save=True, directory='/kaggle/activations')

In [ ]:
plt.imshow(Image.open('path_of_activation_images'))

In [ ]:
!wget -nd -r -P /kaggle/working -A jpg https://mappinglondon.co.uk/wp-content/uploads/2011/02/googlemaps_britishmuseum.jpg

In [ ]:
ls -lah /kaggle/working/googlemaps_britishmuseum.jpg


In [ ]:
!mkdir /kaggle/heatmap

In [ ]:
image = Image.open('image_path')
image = image.resize((256,256))
image_as_array = np.array(image)
image_as_array = image_as_array.astype(np.float32)
ke.display_heatmaps(activations, image_as_array, save=True, directory='/kaggle/heatmap')